# Comparing filters and smoothers

In [ ]:
# import pickle
import os
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import statsmodels.api as sm
import scipy.signal as signal
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from paths import path_samoc, path_results
from filters import lowpass, chebychev, notch, highpass

## filter response

In [ ]:
plt.figure(figsize=(8,5))

b, a = sp.signal.cheby1(4, 1, 100, 'low', analog=True)
w, h = sp.signal.freqs(b, a)
plt.plot(w, 20 * np.log10(abs(h)), label='chebychev')
plt.xscale('log')
plt.title(' frequency response (rp=5)')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')

b, a = sp.signal.butter(4, 100, 'low', analog=True)
w, h = sp.signal.freqs(b, a)
plt.plot(w, 20 * np.log10(abs(h)), label='butterworth')
plt.xscale('log')
plt.title('Butterworth/Chebychev filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')

b, a = sp.signal.butter(4, 100, 'high', analog=True)
w, h = sp.signal.freqs(b, a)
plt.plot(w, 20 * np.log10(abs(h)), label='butterworth')
plt.xscale('log')
plt.title('Butterworth/Chebychev filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')

plt.axhline(0, c='k', lw=.5) # rp
plt.axvline(100, color='r', lw=.5, label='cutoff frequency')

plt.xlim(20,500)
plt.ylim(-50,10)
plt.legend()
plt.tight_layout()

### notch filter

In [ ]:
test = [np.sin(i*np.pi/12) for i in range(240)]
noise = .5*np.random.rand((len(test)))
trend = np.arange(0,1,1/len(test))
test2 = test+noise
test3 = test+noise+trend

In [ ]:
plt.figure(figsize=(12,5))
plt.axhline(0, c='k', lw=.5)
plt.plot(test, label='sin(t)')
plt.plot(trend, label='trend')
plt.plot(noise, label='noise')
plt.plot(noise+trend, label='noise+trend')
plt.plot(test2, label='sin(t)+noise')
plt.plot(test3, label='sin(t)+noise+trend')
# plt.plot(notch(test,12))
plt.plot(notch(test2,12), label='notch(sin+noise)')
plt.plot(notch(test3,12), label='notch(sin+noise+trend)')
plt.legend()

deseasonalizing example: see SST_PDO.ipynb

## LOWESS smoothing
(https://www.statsmodels.org/dev/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html)
statsmodels notes:
Suppose the input data has N points. The algorithm works by estimating the smooth y_i by taking the frac*N closest points to (x_i,y_i) based on their x values and estimating y_i using a weighted linear regression. The weight for (x_j,y_j) is tricube function applied to abs(x_i-x_j).

Stolpe et al. (2017): "To suppress high-frequency variability, we smooth
our results by applying local regressions using locally weighted scatterplot smoothing (LOESS) with a second- order polynomial (Cleveland 1979). Each regression is calculated using 23 years of data."

In [ ]:
lowess = sm.nonparametric.lowess

In [ ]:
x = np.random.uniform(low = -2*np.pi, high = 2*np.pi, size=500)
y = np.sin(x) + np.random.normal(size=len(x))
z = lowess(y, x)
w = lowess(y, x, frac=1./10)

plt.scatter(x, y)
plt.plot(z[:,0], z[:,1], c='r', lw=3)
plt.plot(w[:,0], w[:,1], c='orange', lw=3)

## comparing LOWESS, Butterworth, and Chebychev

In [ ]:
n = 200
t = np.arange(0,200)
periods = [8,16,24,32,64,128,256]
f, ax = plt.subplots(len(periods),1, sharex=True, figsize=(8,12))
for i, p in enumerate(periods):
    y = np.sin(2*np.pi*t/p)
    ylp = lowpass(y[25:-25], 13)
    yhp = highpass(y[25:-25], 100)
    ycb = chebychev(y[25:-25], 13)
    z = lowess(y[25:-25], t[25:-25], frac=1/6.)  # approx 23
    
    ax[i].text(-5, .8, f'period={p}')
    ax[i].plot(t, y)
    ax[i].plot(t[25:-25], ylp, label='lowpass')
    ax[i].plot(t[25:-25], yhp, label='highpass')
    ax[i].plot(t[25:-25], ycb, label='chebychev')
    ax[i].plot(z[:,0], z[:,1], label='LOWESS')
    ax[i].fill([25,32,32,25],[-1,-1,1,1], alpha=.2)
ax[i].legend()
plt.tight_layout()

# High pass filter

# Lanczos filter

In [ ]:
da = xr.DataArray(data=np.random.rand((100000)), dims='time', coords={'time':np.arange(100000)})
Lda = Lanczos(da)
lda = lowpass(da, 120)

f, ax = plt.subplots(1,2, figsize=(12,4))
ax[0].plot(da)
ax[0].plot(Lda)
ax[0].plot(lda)

ATS(da).plot_spectrum(ax=ax[1])
ATS(Lda).plot_spectrum(ax=ax[1])
ATS(lda).plot_spectrum(ax=ax[1])
ax[1].axvline(1/120)